In [1]:
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp, Operator
import qiskit.quantum_info as qi 
from qiskit.circuit import QuantumCircuit
from qiskit.utils import algorithm_globals
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from qiskit import Aer, BasicAer
from qiskit.utils import QuantumInstance
import multiprocessing as multip
from numpy import linalg as LA
import os
from multiprocessing import Pool, TimeoutError
import time
from joblib import Parallel, delayed

num_q = 10
J = 1
Delta = 1.5
List_1 = []
op = ""
for i in range(num_q-1):
    op = ""
    for k in range(i):
        op += "I"
    op += "ZZ"
    for k in range(i+1,num_q-1):
        op += "I"
    List_1.append(op)

List_1.reverse()
H1 = SparsePauliOp(List_1, J*np.ones(num_q-1))  

List_2 = []
op = ""
for i in range(num_q):
    op = ""
    for k in range(i):
        op += "I"
    op += "X"
    for k in range(i+1,num_q):
        op += "I"
    List_2.append(op)
H2 = SparsePauliOp(List_2, Delta*np.ones(num_q))

H = SparsePauliOp.sum([H1,H2])

# Ordering in SparsePauliOp e.g. IIIIIIZZ is the measurement of Z gates on the first two qubits and so forth.

Hmat = Operator(H)
Hmat = Hmat.data

e, v = LA.eig(Hmat)
min_index = np.argmin(e)
v_min = v[:,min_index]
ground_e = np.min(e)
print(H[0])
print(np.min(e))
print(H.size)
print(H.paulis[0])


SparsePauliOp(['IIIIIIIIZZ'],
              coeffs=[1.+0.j])
(-16.535254946759277+0j)
19
IIIIIIIIZZ


In [2]:


"""circuit construction"""

layer = 18
num_p = (2*num_q-1)*layer

weights = ParameterVector("weight",num_p)


def circuit_ex(weights):
    circ = QuantumCircuit(num_q, num_q)
    
    for j in range(num_q):
        circ.ry(np.pi/2*3,j)
        
    for i in range(layer):
        
        for j in range(int(num_q/2)):
            circ.rzz(weights[j+(2*num_q-1)*i],2*j,2*j+1)
        for j in range(int(num_q/2)-1):
            circ.rzz(weights[j+int(num_q/2)+(2*num_q-1)*i], 2*j+1, 2*j+2)
        for j in range(num_q):
            circ.rx(weights[j+num_q-1+(2*num_q-1)*i], j)
        
        
    return circ


In [1]:

num_p_real = 2*layer

# load trajectory data
mat = scipy.io.loadmat('RCD_weight_1.mat')
ws = mat['rcdm_weight']
ws = ws[:,60:65]
identity = np.identity(num_p)


index = []
count = -1
for i in range(num_p_real):
    for j in range(i,num_p_real):
        count = count + 1 
        index.append([i,j,count])
        
index = np.asarray(index)
print(index.shape)


def weight_converter(WEIGHTS):
    weight_long = []
    for j in range(2*layer):
        if j % 2 == 0:
            for i in range(num_q-1):
                weight_long.append(WEIGHTS[j])
        else:
            for i in range(num_q):
                weight_long.append(WEIGHTS[j])
    
    return weight_long



def expectation_loss(WEIGHTS):
    qc = circuit_ex(WEIGHTS)
    qc.save_statevector()
    qc = transpile(qc, simulator)
    result = simulator.run(qc).result()
    state_vector = result.get_statevector(qc)
    psi = np.asarray(state_vector)
    Hpsi = Hmat.dot(psi)
    expectation = np.inner(np.conjugate(psi),Hpsi)
    return np.real(expectation)

    
def f_pospos(WEIGHTS,k):
    WEIGHT_long = weight_converter(WEIGHTS)
    i = index[k,0]
    j = index[k,1]
    f_pp = 0
    if i % 2 == 0:
        INDEX_i = int(i/2)*(2*num_q-1)
        range_i = num_q-1
    else:
        INDEX_i = int(i/2)*(2*num_q-1)+num_q-1
        range_i = num_q
    if j % 2 == 0:
        INDEX_j = int(j/2)*(2*num_q-1)
        range_j = num_q-1
    else:
        INDEX_j = int(j/2)*(2*num_q-1)+num_q-1
        range_j = num_q

    for ii in range(range_i):
        for jj in range(range_j):
            direction_i = np.pi/2 *identity[:,INDEX_i+ii]
            direction_j = np.pi/2 *identity[:,INDEX_j+jj]
            W_pp = WEIGHT_long+direction_i.flatten()+direction_j.flatten()
            u = qi.Statevector.from_instruction(circuit_ex(W_pp))
            Hu = Hmat.dot(u)
            f_pp += 0.5*np.real(np.inner(np.conjugate(u),Hu))
            
    return f_pp

def f_posneg(WEIGHTS,k):
    WEIGHT_long = weight_converter(WEIGHTS)
    i = index[k,0]
    j = index[k,1]
    f_pm = 0
    if i % 2 == 0:
        INDEX_i = int(i/2)*(2*num_q-1)
        range_i = num_q-1
    else:
        INDEX_i = int(i/2)*(2*num_q-1)+num_q-1
        range_i = num_q
    if j % 2 == 0:
        INDEX_j = int(j/2)*(2*num_q-1)
        range_j = num_q-1
    else:
        INDEX_j = int(j/2)*(2*num_q-1)+num_q-1
        range_j = num_q

    for ii in range(range_i):
        for jj in range(range_j):
            direction_i = np.pi/2 *identity[:,INDEX_i+ii]
            direction_j = np.pi/2 *identity[:,INDEX_j+jj]
            W_pm = WEIGHT_long+direction_i.flatten()-direction_j.flatten()
            u = qi.Statevector.from_instruction(circuit_ex(W_pm))
            Hu = Hmat.dot(u)
            f_pm += 0.5*np.real(np.inner(np.conjugate(u),Hu))
            
    return f_pm


def f_negpos(WEIGHTS,k):
    WEIGHT_long = weight_converter(WEIGHTS)
    i = index[k,0]
    j = index[k,1]
    f_mp = 0
    if i % 2 == 0:
        INDEX_i = int(i/2)*(2*num_q-1)
        range_i = num_q-1
    else:
        INDEX_i = int(i/2)*(2*num_q-1)+num_q-1
        range_i = num_q
    if j % 2 == 0:
        INDEX_j = int(j/2)*(2*num_q-1)
        range_j = num_q-1
    else:
        INDEX_j = int(j/2)*(2*num_q-1)+num_q-1
        range_j = num_q

    for ii in range(range_i):
        for jj in range(range_j):
            direction_i = np.pi/2 *identity[:,INDEX_i+ii]
            direction_j = np.pi/2 *identity[:,INDEX_j+jj]
            W_mp = WEIGHT_long-direction_i.flatten()+direction_j.flatten()
            u = qi.Statevector.from_instruction(circuit_ex(W_mp))
            Hu = Hmat.dot(u)
            f_mp += 0.5*np.real(np.inner(np.conjugate(u),Hu))
            
    return f_mp


def f_negneg(WEIGHTS,k):
    WEIGHT_long = weight_converter(WEIGHTS)
    i = index[k,0]
    j = index[k,1]
    f_mm = 0
    if i % 2 == 0:
        INDEX_i = int(i/2)*(2*num_q-1)
        range_i = num_q-1
    else:
        INDEX_i = int(i/2)*(2*num_q-1)+num_q-1
        range_i = num_q
    if j % 2 == 0:
        INDEX_j = int(j/2)*(2*num_q-1)
        range_j = num_q-1
    else:
        INDEX_j = int(j/2)*(2*num_q-1)+num_q-1
        range_j = num_q

    for ii in range(range_i):
        for jj in range(range_j):
            direction_i = np.pi/2 *identity[:,INDEX_i+ii]
            direction_j = np.pi/2 *identity[:,INDEX_j+jj]
            W_mm = WEIGHT_long-direction_i.flatten()-direction_j.flatten()
            u = qi.Statevector.from_instruction(circuit_ex(W_mm))
            Hu = Hmat.dot(u)
            f_mm += 0.5*np.real(np.inner(np.conjugate(u),Hu))
            
    return f_mm

print(ws.shape)



num_entry = index[:,0].size
num_traj = 10
num_result = ws.shape[1]

PP = []
pp_mat = np.empty((0,num_result), int)

PM = []
pm_mat = np.empty((0,num_result), int)

MP = []
mp_mat = np.empty((0,num_result), int)

MM = []
mm_mat = np.empty((0,num_result), int)




for j in range(num_traj):
    print(j)
    PP = []
    PM = []
    MP = []
    MM = []
    for i in range(num_result):
        weights = ws[num_p_real*j:num_p_real*(j+1),i]
        start = time.time()
        results_pp = Parallel(n_jobs=multip.cpu_count())(delayed(f_pospos)(weights,i) for i in range(num_entry))
        results_pm = Parallel(n_jobs=multip.cpu_count())(delayed(f_posneg)(weights,i) for i in range(num_entry))
        results_mp = Parallel(n_jobs=multip.cpu_count())(delayed(f_negpos)(weights,i) for i in range(num_entry))
        results_mm = Parallel(n_jobs=multip.cpu_count())(delayed(f_negneg)(weights,i) for i in range(num_entry))
        end = time.time()
        print(end-start)
        PP.append(results_pp)
        PM.append(results_pm)
        MP.append(results_mp)
        MM.append(results_mm)
    pp = np.asarray(PP)
    pm = np.asarray(PM)
    mp = np.asarray(MP)
    mm = np.asarray(MM)
    pp_mat = np.append(pp_mat, np.transpose(pp), axis = 0)
    pm_mat = np.append(pm_mat, np.transpose(pm), axis = 0)
    mp_mat = np.append(mp_mat, np.transpose(mp), axis = 0)
    mm_mat = np.append(mm_mat, np.transpose(mm), axis = 0)
        
    

scipy.io.savemat('pp_rcd_13.mat', {'pp': pp_mat})
scipy.io.savemat('pm_rcd_13.mat', {'pm': pm_mat})
scipy.io.savemat('mp_rcd_13.mat', {'mp': mp_mat})
scipy.io.savemat('mm_rcd_13.mat', {'mm': mm_mat})


NameError: name 'layer' is not defined

In [4]:
mat = scipy.io.loadmat('RCD_weight_1.mat')
ws = mat['rcdm_weight']
print(ws.shape)

(360, 110)


In [19]:
print(pp_mat.shape)
print(pp.shape)

pp_mat = np.append(pp_mat, pp, axis = 0)

(1332, 2)
(2, 666)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 2 and the array at index 1 has size 666

In [15]:
print(pp_mat.shape)
print(pp.shape)
print(pm.shape)
print(len(PP))
print(mp.cpu_count())

(0, 1)
(1, 666)
(1, 666)
1


AttributeError: 'numpy.ndarray' object has no attribute 'cpu_count'

In [20]:
666*22

14652